# CR_Score Playbook 02: Feature Selection

**Level:** Intermediate  
**Time:** 15-20 minutes  
**Goal:** Master model-agnostic feature selection methods

## What You'll Learn

- Forward selection (greedy addition)
- Backward elimination (greedy removal)
- Stepwise selection (bidirectional)
- Exhaustive search (all combinations)
- MLflow experiment tracking
- Compare results across methods

## Prerequisites

- Completed Playbook 01
- MLflow installed: `pip install mlflow`

## Step 1: Setup

In [1]:
import pandas as pd
import numpy as np
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root / 'src'))

from cr_score.model import LogisticScorecard
from cr_score.features import ForwardSelector, BackwardSelector, StepwiseSelector

print("[OK] Libraries imported!")

[OK] Libraries imported!


In [ ]:
# Load data
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

# Separate features and target
# NOTE: For this tutorial, we'll only use NUMERIC features
# In practice, you'd encode categorical features first
numeric_cols = train_df.select_dtypes(include=['int64', 'float64']).columns.tolist()
feature_cols = [col for col in numeric_cols 
                if col not in ['application_id', 'default']]

X_train = train_df[feature_cols]
y_train = train_df['default']
X_test = test_df[feature_cols]
y_test = test_df['default']

print(f"Training data: {len(X_train)} samples, {len(feature_cols)} numeric features")
print(f"Features: {feature_cols}")
print(f"Test data: {len(X_test)} samples")

Training data: 3500 samples, 14 features
Test data: 1500 samples


## Step 2: Forward Selection

Start with no features, add best one at a time.

In [3]:
# Create estimator (model to use for selection)
# Using CR_Score's LogisticScorecard instead of sklearn's LogisticRegression
estimator = LogisticScorecard(random_state=42)

# Create forward selector
forward = ForwardSelector(
    estimator=estimator,
    max_features=8,
    use_mlflow=False  # Disable MLflow for simplicity
)

# Fit (this will take a minute as it evaluates many feature combinations)
print("Running forward selection... (this may take 1-2 minutes)")
forward.fit(X_train, y_train)

# Get selected features
selected_features = forward.get_selected_features()

print(f"\nForward Selection Results:")
print(f"  Selected {len(selected_features)} features")
print(f"  Features: {selected_features}")
print(f"  Best AUC: {forward.best_score_:.3f}")

Running forward selection... (this may take 1-2 minutes)
{"n_features": 14, "max_features": 8, "scoring": "roc_auc", "event": "Starting forward feature selection", "timestamp": "2026-01-16T10:08:00.807621Z", "level": "info"}
{"n_samples": 2800, "n_features": 1, "weighted": false, "event": "Fitting logistic regression model", "timestamp": "2026-01-16T10:08:00.811622Z", "level": "info"}
{"intercept": -7.5984166742411565, "n_features_used": 1, "event": "Model training completed", "timestamp": "2026-01-16T10:08:00.820143Z", "level": "info"}
{"n_samples": 2800, "n_features": 1, "weighted": false, "event": "Fitting logistic regression model", "timestamp": "2026-01-16T10:08:00.861168Z", "level": "info"}
{"intercept": -8.072470435139289, "n_features_used": 1, "event": "Model training completed", "timestamp": "2026-01-16T10:08:00.868334Z", "level": "info"}
{"n_samples": 2800, "n_features": 1, "weighted": false, "event": "Fitting logistic regression model", "timestamp": "2026-01-16T10:08:00.8703

c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:945: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 166, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 409, 

{"intercept": -5.963478815945979, "n_features_used": 1, "event": "Model training completed", "timestamp": "2026-01-16T10:08:00.990404Z", "level": "info"}
{"n_samples": 2800, "n_features": 1, "weighted": false, "event": "Fitting logistic regression model", "timestamp": "2026-01-16T10:08:00.993607Z", "level": "info"}
{"intercept": -6.5009998272878615, "n_features_used": 1, "event": "Model training completed", "timestamp": "2026-01-16T10:08:01.001088Z", "level": "info"}
{"n_samples": 2800, "n_features": 1, "weighted": false, "event": "Fitting logistic regression model", "timestamp": "2026-01-16T10:08:01.003594Z", "level": "info"}
{"intercept": -6.498894160129902, "n_features_used": 1, "event": "Model training completed", "timestamp": "2026-01-16T10:08:01.009376Z", "level": "info"}
{"n_samples": 2800, "n_features": 1, "weighted": false, "event": "Fitting logistic regression model", "timestamp": "2026-01-16T10:08:01.011386Z", "level": "info"}
{"intercept": -6.441937581742946, "n_features_us

c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\model_selection\_validation.py:945: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 166, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 409, in _score
    y_pred = method_caller(
             ^^^^^^^^^^^^^^
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\metrics\_scorer.py", line 96, in _cached_call
    result, _ = _get_response_values(
                ^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\utils\_response.py", line 235, in _get_response_values
    rai

ValueError: 
All the 5 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\model_selection\_validation.py", line 833, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\src\cr_score\model\logistic.py", line 106, in fit
    self.model_.fit(X.values, y.values, sample_weight=weights)
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv\Lib\site-packages\sklearn\base.py", line 1336, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\EDMUN\workspace\projects\21_ScoreForge\.venv

## Step 3: Backward Elimination

Start with all features, remove worst one at a time.

In [ ]:
# Create estimator
estimator = LogisticScorecard(random_state=42)

# Create backward selector
backward = BackwardSelector(
    estimator=estimator,
    min_features=5,
    use_mlflow=False
)

# Fit
print("Running backward elimination... (this may take 1-2 minutes)")
backward.fit(X_train, y_train)

# Get selected features
selected_features = backward.get_selected_features()

print(f"\nBackward Elimination Results:")
print(f"  Selected {len(selected_features)} features")
print(f"  Features: {selected_features}")
print(f"  Best AUC: {backward.best_score_:.3f}")

## Step 4: Stepwise Selection

Bidirectional: can add or remove features.

In [ ]:
# Create estimator
estimator = LogisticScorecard(random_state=42)

# Create stepwise selector
stepwise = StepwiseSelector(
    estimator=estimator,
    max_features=8,
    use_mlflow=False
)

# Fit
print("Running stepwise selection... (this may take 1-2 minutes)")
stepwise.fit(X_train, y_train)

# Get selected features
selected_features = stepwise.get_selected_features()

print(f"\nStepwise Selection Results:")
print(f"  Selected {len(selected_features)} features")
print(f"  Features: {selected_features}")
print(f"  Best AUC: {stepwise.best_score_:.3f}")

## Step 5: Compare Methods

Let's compare all three methods.

In [ ]:
# Compare results
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 6))

methods = ['Forward', 'Backward', 'Stepwise']
aucs = [forward.best_score_, backward.best_score_, stepwise.best_score_]
n_features = [
    len(forward.get_selected_features()),
    len(backward.get_selected_features()),
    len(stepwise.get_selected_features())
]

x = np.arange(len(methods))
width = 0.35

ax.bar(x - width/2, aucs, width, label='AUC', color='skyblue')
ax.bar(x + width/2, [n/10 for n in n_features], width, 
       label='# Features / 10', color='lightcoral')

ax.set_xlabel('Method')
ax.set_ylabel('Score')
ax.set_title('Feature Selection Method Comparison')
ax.set_xticks(x)
ax.set_xticklabels(methods)
ax.legend()
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nSummary:")
for method, auc, n_feat in zip(methods, aucs, n_features):
    print(f"  {method:12s}: AUC={auc:.3f}, Features={n_feat}")

## Summary

You learned how to:
- Apply forward selection (greedy addition)
- Apply backward elimination (greedy removal)
- Apply stepwise selection (bidirectional)
- Compare different methods

**Next:** Playbook 03 for visualization and reporting!